# Programmazione ad oggetti

## Cenni sulle classi

per definire una classe si utilizza la keyword `class` <NomeClasse>:
    
```python
class MiaClasse:
    # __init__ è il costruttore, viene eseguito quando viene creato l'oggetto
    # self è la keyword che permette di accedere agli altri metodi della classe
    # deve essere specificata obbligatoriamente come primo parametro della funzione
    def __init__(self):
        pass
    
    def miafunzione(self):
        return "miafunzione"
    
    def mia_funzione_params(self, parametro):
        return "ciao " + parametro
```
    
per creare un nuovo oggetto di tipologia MiaClasse si usa chiamare la classe attraverso le parenetesi, come se fosse una funzione.
    
```python
m = MiaClasse()

```

## Definizione della classe Persona

attraverso la classe Persona, possiamo istanziare oggetti che rappresentano una persona.

attraverso il costruttore `__init__` richiediamo due argomenti che andranno specificati quando si creerà un nuovo oggetto di tipologia Persona

In [182]:
class Persona:
    
    def __init__(self, nome, cognome):
        self.nome = nome
        self.cognome = cognome
        
    # metodo che ci permette di ritornare il nome completo della persona
    # attraverso l'uninone delle proprietà nome e cognome
    def nome_completo(self):
        return f"{self.nome} {self.cognome}"
    
    # permette di stampare a scherom un determinato messaggio
    # nel formato:
    # Nome Completo> messaggio
    def parla(self, message):
        print(f"{self.nome_completo()}> {message}")
        

# creazione di due oggetti Persona
mario = Persona("mario", "rossi")
giacomo = Persona("giacomo", "bianchi")

mario.parla("ciao")
giacomo.parla("ciao")


mario rossi> ciao
giacomo bianchi> ciao


## Ereditarietà

L'ereditarietà ci permette di definire una classe padre e delle classe figlie che ereditano tutti  i metodi e proprietà della classe genitrice.

nel seguente esempio andiamo a definire un oggetto denominato Auto che rappresenta un automobile.

le funzioni definite all'interno della classe vengono chiamate `metodi` dell'oggetto, mentre le variabili vengono definite `proprietà` dell'oggetto.

In [129]:
import uuid

import logging

logging.basicConfig(level=logging.DEBUG)
logger_auto = logging.getLogger("Auto")

class Auto:
    
    MOTORE_SPENTO = 0
    MOTORE_ACCESO = 1
    
    PREZZO_BASE = 0
    
    ## costruttore
    def __init__(self, *args, **kwargs):
        if 'prezzo' not in kwargs:
            kwargs['prezzo'] = self.PREZZO_BASE
        for key, val in kwargs.items():
            setattr(self, key, val)
        self.stato_motore = self.MOTORE_SPENTO
        self.accessori = {}

        # generiamo un nuovo serial number per ogni macchina
        self.numero_seriale = uuid.uuid4().hex
        
        self._accessori_base()
        for key, item in  self.accessori.items():
            logger_auto.debug("aggiunto accessorio")
            self.prezzo += item.prezzo
            
    # viene implementato nelle classi figlie
    def _accessori_base(self):
        pass
        
    def accendi_motore(self):
        self.stato_motore = self.MOTORE_ACCESO

    def spegni_motore(self):
        self.stato_motore = self.MOTORE_SPENTO
        
    def aggiungi_accessorio(self, *args, **kwargs):
        for key, val in kwargs.items():
            if key not in self.accessori:
                logger_auto.debug("aggiunta accessorio")
                self.accessori[key] = val
                self.prezzo += val.prezzo
                
    def aggiungi_accessori(self, *args, **kwargs):
        for key, item in kwargs.items():
            self.aggiungi_accessorio(**{ key : item})
                
    def modifica_accessorio(self, *args, **kwargs):
        for key, val in kwargs.items():
            self.accessori[key] = val
            
    def rimuovi_accessorio(self, *args, **kwargs):
        for key, val in kwargs.items():
            if key in self.accessori:
                del(self.accessori[key])
                self.prezzo -= val.prezzo
        
        
class Accessorio:
    
    def __init__(self, nome: str, descrizione: str, prezzo: float):
        self.nome = nome
        self.descrizione = descrizione
        self.prezzo = float(prezzo)
        
        
# Definizione della classe che rappresenta il modello di auto.
# estende la classe genitrice Auto e ne implementa tutti i metodi.
class Cinquecento(Auto):
    
    PREZZO_BASE = 14000
    
    def __init__(self, *args, **kwargs):
        # il prezzo finale della macchina comprende anche il costo degli accessori
        super().__init__(marca="Fiat", modello=self.__class__.__name__, **kwargs)

class Giulietta(Auto):
    
    PREZZO_BASE = 15000
    
    # proprietà della classe, in condivisione con tutte le istanze
    # se si creano N  oggetti di tipologia  Giulietta, la modifica di `accessori` ad un istanza 
    # comporta la modifica per tutti gli oggetti
    # reference: https://docs.python.org/3/tutorial/classes.html#class-objects
    
    def __init__(self, *args, **kwargs):
        # il prezzo finale della macchina comprende anche il costo degli accessori
        super().__init__(marca="AlfaRomeo", modello=self.__class__.__name__, **kwargs)
        
    def _accessori_base(self):
        self.aggiungi_accessorio(radio=Accessorio("radio",None, 300))
        self.aggiungi_accessorio(navigatore=Accessorio("navigatore",None, 1500))


g = Giulietta(prezzo=15000)
c = Cinquecento(prezzo=15000)
print(g.__dict__)
print(c.__dict__)
        

DEBUG:Auto:aggiunta accessorio
DEBUG:Auto:aggiunta accessorio
DEBUG:Auto:aggiunto accessorio
DEBUG:Auto:aggiunto accessorio


{'marca': 'AlfaRomeo', 'modello': 'Giulietta', 'prezzo': 18600.0, 'stato_motore': 0, 'accessori': {'radio': <__main__.Accessorio object at 0x7f88b80e5ad0>, 'navigatore': <__main__.Accessorio object at 0x7f88c02bef50>}, 'numero_seriale': '02e256bd10704955aa05b3ce9563c43a'}
{'marca': 'Fiat', 'modello': 'Cinquecento', 'prezzo': 15000, 'stato_motore': 0, 'accessori': {}, 'numero_seriale': 'b4d988b6093c47e2b99c9f9b1ca3c25f'}


In [111]:
import logging

logger = logging.getLogger("Garage")

class Garage:
    
    def __init__(self, capienza_max: int = 40):
        self.capienza_max = capienza_max
        self.automobili = []
        
    def posti_disponibili(self):
        return len(self.automobili) + 1 <= self.capienza_max
        
    def aggiungi_auto(self, auto: Auto):
        if len(self.automobili) + 1 < self.capienza_max:
            if not self.ricerca_auto(auto):
                self.automobili.append(auto)
            else:
                logger.debug("auto già presente nel garage")
        else:
            logger.debug("garage pieno, impossibile aggiungere auto")
            
    def ricerca_auto(self, auto: Auto) -> Auto:
        for item in self.automobili:
            if item.numero_seriale ==  auto.numero_seriale:
                return item
        return None
            
    def rimuovi_auto(self, auto: Auto):
        auto = self.ricerca_auto(auto)
        if auto:
            self.automobili.remove(auto)
            logger.debug("auto rimossa")
        else:
            logger.debug("auto non presente. impossibile rimuovere")

    def calcola_valore_magazzino(self):
        count = 0.0
        for item in self.automobili:
            count += item.prezzo
        return count

In [135]:
g = Garage()

giu1 = Giulietta()
giu2 = Giulietta(prezzo=21000, motore=1600)
ci1 = Cinquecento()

accessori = {
    "sensore_posteriore" : Accessorio("sensori_parcheggio_posteriori", "Sensori Parcheggio Posteriori", 300),
    "sensore_anteriore" : Accessorio("sensori_parcheggio_anteriori", "Sensori Parcheggio Anteriori", 300)
}
giu2.aggiungi_accessori(**accessori)

g.aggiungi_auto(giu1)
g.aggiungi_auto(giu2)
g.aggiungi_auto(ci1)

print("valore magazzino: {}".format(g.calcola_valore_magazzino()))
for item in g.automobili:
    print(item.marca, item.modello, item.numero_seriale, item.prezzo, [k for k, v in item.accessori.items()])

DEBUG:Auto:aggiunta accessorio
DEBUG:Auto:aggiunta accessorio
DEBUG:Auto:aggiunto accessorio
DEBUG:Auto:aggiunto accessorio
DEBUG:Auto:aggiunta accessorio
DEBUG:Auto:aggiunta accessorio
DEBUG:Auto:aggiunto accessorio
DEBUG:Auto:aggiunto accessorio
DEBUG:Auto:aggiunta accessorio
DEBUG:Auto:aggiunta accessorio


valore magazzino: 57800.0
AlfaRomeo Giulietta f90649ff53794b76b5de9e40d260ab3f 18600.0 ['radio', 'navigatore']
AlfaRomeo Giulietta 7d61ac9fc8bc453d9eca6530f8376ac7 25200.0 ['radio', 'navigatore', 'sensore_posteriore', 'sensore_anteriore']
Fiat Cinquecento 5126aad6f8f646dca6940453b7b9a0b5 14000 []


INFO:ContoCorrente:richiesta apertura conto corrente a nome di Mario Rossi
INFO:ContoCorrente:richiesta fido
INFO:ContoCorrente:richiesta fido approvata: 2500€
DEBUG:ContoCorrente:conto corrente creato
INFO:ContoCorrente:richiesta di deposito 10000


saldo disponibile: 5500
fido: 2500
saldo: 3000
saldo: 13000
